In [ ]:
import bw2data as bd
import bw2calc as bc
import multifunctional as mf

# Create a basic database with a separate product

Note the node type `multifunctional` and the edge attribute `functional`.

In [2]:
if "multifunctional-products-example" in bd.projects:
    bd.projects.delete_project("multifunctional-products-example", True)

In [3]:
bd.projects.set_current("multifunctional-products-example")

In [4]:
PRODUCT_DATA = {
    ("products", "a"): {
        "name": "flow - a",
        "code": "a",
        "unit": "kg",
        "type": "emission",
        "categories": ("air",),
    },
    ("products", "product"): {
        "type": "product",
        "name": "first product",
        "unit": "kg",
        "exchanges": [],
    },
    ("products", "1"): {
        "name": "process - 1",
        "code": "1",
        "location": "first",
        "type": "multifunctional",
        "exchanges": [
            {
                "functional": True,
                "type": "production",
                "input": ("products", "product"),
                "amount": 4,
                "properties": {
                    "price": 7,
                    "mass": 6,
                },
            },
            {
                "functional": True,
                "type": "production",
                "name": "second product - 1",
                "unit": "megajoule",
                "amount": 6,
                "properties": {
                    "price": 12,
                    "mass": 4,
                },
            },
            {
                "type": "biosphere",
                "name": "flow - a",
                "amount": 10,
                "input": ("products", "a"),
            },
        ],
    },
}

db = mf.MultifunctionalDatabase("products")
db.register(default_allocation="price")
db.write(PRODUCT_DATA)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 12192.74it/s]
2024-07-07 08:20:41.160 | DEBUG    | multifunctional.node_classes:allocate:68 - Allocating Multifunctional: 'process - 1' (None, first, None) (id: 3) with strategy price
2024-07-07 08:20:41.161 | DEBUG    | multifunctional.allocation:generic_allocation:56 - Using allocation factor 0.28 for functional edge {'functional': True, 'type': 'production', 'input': ('products', 'product'), 'amount': 4, 'properties': {'price': 7, 'mass': 6}, 'output': ('products', '1')} on activity {'name': 'process - 1', 'code': '1', 'location': 'first', 'type': 'multifunctional', 'database': 'products', 'id': 3, 'exchanges': [{'functional': True, 'type': 'production', 'input': ('products', 'product'), 'amount': 4, 'properties': {'price': 7, 'mass': 6}, 'output': ('products', '1')}, {'functional': True

Vacuuming database 


Writing the database also allocates the multifunctional process:

In [5]:
for node in bd.Database("products"):
    print(node, node['type'])

Read-only allocated process: 'process - 1' (megajoule, first, None) readonly_process
Read-only allocated process: 'process - 1' (kg, first, None) readonly_process
'first product' (kg, None, None) product
'flow - a' (kg, None, ('air',)) emission
Multifunctional: 'process - 1' (None, first, None) multifunctional


Links to the `product` node are preserved:

In [6]:
for exc in bd.get_node(name="process - 1", unit=None, database="products").production():
    print(exc)

Exchange: 4 kg 'first product' (kg, None, None) to Multifunctional: 'process - 1' (None, first, None)>
Exchange: 6 megajoule Read-only allocated process: 'process - 1' (megajoule, first, None) to Multifunctional: 'process - 1' (None, first, None)>


The product node can be used in LCIA:

In [7]:
m = bd.Method(("foo",))
m.register()
m.write([(bd.get_node(code="a").id, 5)])

In [8]:
fu, objs, _ = bd.prepare_lca_inputs(
    demand={bd.get_node(code="product"): 1}, method=("foo",)
)
lca = bc.LCA(fu, data_objs=objs)
lca.lci()
lca.lcia()
lca.score

3.4999999403953552